<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/dev_branch_EDA_Prediction/4_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
reset -f


**Import der Libraries und Verbindung mit Google Drive und Sqlite3**

In [ ]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas_profiling import ProfileReport
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def connect_to_db(db_file):
    sqlite3_conn = None
    try:
        sqlite3_conn = sq.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()
  
all_conn = sqlite3.connect('/content/drive/Shareddrives/Projektarbeit DMA Gruppe D/DWH_dbs/DWH_allergy_test.db')


In [ ]:
# create a temporary empty all_database in the memory
all_cur = all_conn.cursor()

In [ ]:
# List of tables in all_database
all_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(all_cur.fetchall())

In [ ]:
# alle Spalten
predallobs_df = pd.read_sql('''SELECT * FROM observations_info''', all_conn)
predallobs_df.columns

**Erstellen eines Ursprungsdataframes aus dem Datawarehouse**




In [ ]:
lefjoinmedpat_df = pd.read_sql_query("""
SELECT * FROM medications_info LEFT JOIN patients_info ON medications_info.PATIENT = patients_info.Id
""", all_conn
  )

In [ ]:
lefjoinmedpat_df.head(3)

In [ ]:
# Removing Duplicate Rows
print("Number of Duplicated Rows", lefjoinmedpat_df.duplicated(lefjoinmedpat_df.columns).sum())

In [ ]:
lefjoinmedpat_df.shape

In [ ]:
#Entfernen von Duplikaten
lefjoinmedpat_df.drop_duplicates(keep="first",inplace=True) 
print("Size of dataset after removing duplicated rows", lefjoinmedpat_df.shape)

In [ ]:
# Übersicht 
lefjoinmedpat_df.nunique(axis=0)

In [ ]:
#Ursprung der Daten
lefjoinmedpat_df.groupby(["RACE"]).size()

**Erstellen eines Dataframes, der nur Angaben zu COVID-positiven Patienten enthält. Diese werden in einer zusätzlichen Spalte mit "1" bezeichnet**





In [ ]:
#Zunächst Selektion der positiven Testergebnisse
postest_df = predallobs_df.loc[(predallobs_df["CODE"] == "94531-1") & (predallobs_df["VALUE"] == "Detected (qualifier value)")] 
postest_df.head(3)



In [ ]:
# Erstellen eines Dataframes mit allen Laborwerten der Patienten mit positivem Testergebnis
allerg_postest_df = predallobs_df.loc[((predallobs_df.PATIENT.isin(postest_df['PATIENT'])))]
allerg_postest_df.head(10)


In [ ]:
predallcon_df = pd.read_sql('''SELECT * FROM conditions_info''', all_conn)
predallobs_df.columns

In [ ]:
predallcon_df.groupby(["DESCRIPTION"]).size()

In [ ]:
# Löschen der Patienten mit bestimmten Spaltenwerten / #Using drop() to delete rows based on column value
# df.drop(df[df['Fee'] >= 24000].index, inplace = True)
droptubalpregnancy_df = predallcon_df.drop(predallcon_df[predallcon_df['DESCRIPTION'] == "Tubal pregnancy"].index)

In [ ]:
droptubalpregnancy_df.groupby(["DESCRIPTION"]).size()

In [ ]:
# Hinzufügen einer zusätzlichen Spalte mit dem Wert "1"
predallcon_df['datenbankart'] = 'asthma'
predallcon_df.head(3)

In [ ]:
#Übersicht der Condition-Kohorte
predallcon_df.nunique(axis=0)

**Erstellen eines Dataframes, der nur die Patienten der Kontrollgruppe beinhaltet. Diese werden in einer zusätzlichen Spalte mit "0" bezeichnet**

In [ ]:
#Selektion der Laborparameter von Patienten ohne positiven COVID-Test aus dem Ursprungsdataframe
allergy_negtest_df = predallobs_df.loc[~((predallobs_df.PATIENT.isin(allerg_postest_df['PATIENT'])))]
allergy_negtest_df.head(3)

In [ ]:
allergy_negtest_df.shape

In [ ]:
#Übersicht Kontrolgruppe
allergy_negtest_df.nunique(axis=0)

In [ ]:
# Erstellen eines Dataframes von Patienten mit der Nebendiagnose Covid19 
allergy_cov_Nebendiag_df = pd.read_sql_query("""
select PATIENT, CODE, DESCRIPTION from conditions_info where code in ("840539006")
;""", all_conn
  )


In [ ]:
allergy_cov_Nebendiag_df.head(3)

In [ ]:
# Um Sicherzugehen, dass wirklich nur Covid-neg. Patienten in der Kontrollgruppe sind werden auch Patienten ohne Testergebnis aber mit Nebendiagnose Covid19 bzw. Verdacht auf Covid19 ausgeschlossen.
wirklich_nur_neg_cov_df = allergy_negtest_df.loc[~((allergy_negtest_df.PATIENT.isin(allergy_cov_Nebendiag_df["PATIENT"])))]


In [ ]:
# Hinzufügen Spalte mit Wert "0"
wirklich_nur_neg_cov_df['Covid'] = '0'
wirklich_nur_neg_cov_df.head(3)

In [ ]:
wirklich_nur_neg_cov_df.shape

In [ ]:
#Übersicht Kontrolgruppe
wirklich_nur_neg_cov_df.nunique(axis=0)

**Zusammenfügen von COVID-pos. Kohorte und Kontrollgruppe in einen Dataframe**

In [ ]:
pos_plus_neg_cov_df = pd.concat([allerg_postest_df, wirklich_nur_neg_cov_df])
pos_plus_neg_cov_df.head(3)

In [ ]:
# Löschen der COVID-Testergebnisse, damit der Datentyp der Werte der Spalte "Value" in Floats geändert werden kann (Covid-Status ist trotzdem in Spalte Covid festgehalten)
# pos_plus_neg_cov_df = pos_plus_neg_cov_df.loc[(pos_plus_neg_cov_df["CODE"] != "94531-1")]


# Konvertieren der Laborwerte und des Covid-Status in Zahlenwerte (Floats)
# pos_plus_neg_cov_df["Covid"] = pos_plus_neg_cov_df["Covid"].astype("float")
# pos_plus_neg_cov_df["VALUE"] = pos_plus_neg_cov_df["VALUE"].pos_plus_neg_cov_df("float")

# pos_plus_neg_cov_df.head(3)

In [ ]:
#Konvertieren der Datumformats 
# dffinal["DATE"] = pd.to_datetime(dffinal["DATE"])
# dffinal["DATE"] = dffinal["DATE"].dt.to_period('d')

In [ ]:
# dffinal.head()

**Vergleichende Analyse der Laborparameter zwischen den Gruppen**

In [ ]:
%matplotlib inline
#Plotting der Nierenparameter
GFR= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "33914-3"]
Urea= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "3094-0"]
Protein= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "2885-2"]

g = sns.catplot(data=GFR, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=GFR, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('GFR', weight='bold').set_fontsize('16')

g = sns.catplot(data=Urea, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Urea, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Urea', weight='bold').set_fontsize('16')

g = sns.catplot(data=Protein, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Protein, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Protein', weight='bold').set_fontsize('16')

In [ ]:
#Plotting der Kreislaufparameter
Temperatur= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "8310-5"]
O2= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "2708-6"]
AF= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "9279-1"]
HF= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "8867-4"]
Syst= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "8480-6"]
Dia= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "8462-4"]

g = sns.catplot(data=Temperatur, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Temperatur, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Körpertemperatur', weight='bold').set_fontsize('16')

g = sns.catplot(data=O2, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=O2, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('O2-Sättigung', weight='bold').set_fontsize('16')

g = sns.catplot(data=AF, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=AF, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Atemfrequenz', weight='bold').set_fontsize('16')

g = sns.catplot(data=HF, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=HF, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Herzfrequenz', weight='bold').set_fontsize('16')

g = sns.catplot(data=Syst, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Syst, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Systolischer Blutdruck', weight='bold').set_fontsize('16')

g = sns.catplot(data=Dia, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Dia, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Diastolischer Blutdruck', weight='bold').set_fontsize('16')


In [ ]:
#Plotting der Leberparameter
ALT= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "1742-6"]
AST= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "1920-8"]
AP= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "6768-6"]
Bili= pos_plus_neg_cov_df[pos_plus_neg_cov_df['CODE'] == "1975-2"]

g = sns.catplot(data=ALT, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=ALT, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('ALT', weight='bold').set_fontsize('16')

g = sns.catplot(data=AST, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=AST, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('AST', weight='bold').set_fontsize('16')

g = sns.catplot(data=AP, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=AP, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('AP', weight='bold').set_fontsize('16')

g = sns.catplot(data=Bili, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Bili, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Bilirubin total', weight='bold').set_fontsize('16')

**Vorbereitung der Daten für den Classifier**

In [ ]:
# import python packages for ML models
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

from sklearn.model_selection import  GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
dfpred = pos_plus_neg_cov_df.pivot_table('VALUE', ['PATIENT',"DATE", "Covid"], 'DESCRIPTION')
dfpred.head(3)

In [ ]:
dfpred = dfpred.drop(["Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma", "Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma", "Bilirubin.total [Mass/volume] in Serum or Plasma", 'Body temperature', "Diastolic Blood Pressure", "Estimated Glomerular Filtration Rate", "Oxygen [Partial pressure] in Arterial blood", "Oxygen saturation in Arterial blood","Protein [Mass/volume] in Serum or Plasma","Systolic Blood Pressure", "Urea nitrogen [Mass/volume] in Serum or Plasma" ], axis=1)
dfpred = dfpred.reset_index()
dfpred.rename(columns={"Glomerular filtration rate/1.73 sq M.predicted": 'GFR', "Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma":  "ALT", "Heart rate": "Heart_rate", "Respiratory rate": "Respiratory_rate"}, inplace=True)
dfpred.tail(3)

In [ ]:
dfpred= dfpred.dropna(thresh=4)

In [ ]:
dfpred.groupby(["Covid"]).size()

In [ ]:
#Exkurs: Fallenlassen von Zeilen mit NaN-Werten reduziert die Kontrollgruppe in nicht tolerierbarem Maße (d.h. die vier Parameter wurden so gut wie nie gleichzeitig bestimmt)
dfpnull = dfpred.dropna()
dfpnull.groupby(["Covid"]).size()

#Es muss daher ein Algorithmus zur Vorhersage verwendet werden, der NaN-Werte akzeptiert --> XGBoost

In [ ]:
# Mischen der Daten

dfpred= dfpred.sample(frac=1)

In [ ]:
# Splitten der Daten in train und test Dataframes
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# Ausgabe der Shapes von train und test Dataframes
dfpred_train, dfpred_test = train_test_split(dfpred, test_size=test_ratio, random_state=0)
dfpred_train.shape, dfpred_test.shape

In [ ]:
# Bestimmen der Features und des Zielwertes
X = dfpred_train.drop(["patient_id", "Covid","DATE"], axis=1)
y = dfpred_train["Covid"]

# Zusätzlich Erstellen eines Validation Dataframes
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_ratio, random_state=0)

X_train.shape, X_val.shape

**Vorhersage mit XGBoost**

In [ ]:
# Erstellen des Parameter Grid Search
param_grid = {
    'learning_rate': [0.05, 0.5], #so called `eta` value    
    'max_depth': [2, 5, 10, 15],
    'colsample_bytree': [0.4],
    # "objective" :["reg:tweedie"],
    'gamma':[0],               
    'min_child_weight':[1.5],
    'n_estimators' : [5, 10],                                                                    
    'reg_alpha': [0.75],
    'reg_lambda' : [0.45],
    'subsample' : [0.6],
    'seed' : [42]
}
   
# Erstellen des Models
xgb = XGBClassifier()
# Initiieren des Grid Searches
grid_search_xgb = GridSearchCV(estimator = xgb, 
                               param_grid=param_grid,
                               cv=3,
                               n_jobs=-1,
                               verbose=2)

In [ ]:
# Anpassen der Daten an den Grid Search
grid_search_xgb.fit(X_train, y_train)

In [ ]:
# Beste Parameter nach Grid Search
grid_search_xgb.best_params_

In [ ]:
# Beste Model
best_grid_xgb = grid_search_xgb.best_estimator_

# Evaluiierung für die Train Daten
pred_train_xgb = best_grid_xgb.predict(X_train)
print('Accuracy for train data:', accuracy_score(y_train, pred_train_xgb))  

# Evaluiierung für die Test Daten
pred_test_xgb = best_grid_xgb.predict(X_val)
acc_xgb = round(accuracy_score(y_val, pred_test_xgb), 5)
print('Accuracy for test data:', acc_xgb)

In [ ]:
# Hinzufügen des Prediction Ergebnisses zur Ausgangstabelle
selected_columns = X.columns
x_test = dfpred_test[X.columns]
dfpred_test["XGB_prediction"] = best_grid_xgb.predict(x_test)
dfpred_test.head(5)


In [ ]:
# Erstellen einer Confusion-Matrix
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

y_actu = dfpred_test["Covid"]
y_pred = dfpred_test["XGB_prediction"]
cf_matrix = confusion_matrix(y_actu, y_pred)

classes = ['class A', 'class B']
figure, ax = plot_confusion_matrix(conf_mat = cf_matrix,
                                   show_absolute = True,
                                   show_normed = True,
                                   colorbar = True)
plt.show()

In [ ]:
# Darstellen der inkorrekt klassifizierten Daten
incorrect = dfpred_test[dfpred_test["Covid"] != dfpred_test["XGB_prediction"]]
incorrect.head(30)

In [ ]:
# Darstellen der Feature importance
from xgboost import plot_importance
from matplotlib import pyplot
plot_importance(best_grid_xgb)
pyplot.show()

In [ ]:
# Beispielhaftes Darstellen des Decision Trees für die GFR
from xgboost import plot_tree
plot_tree(best_grid_xgb, num_trees=6)
fig = plt.gcf()
fig.set_size_inches(50, 30)